# SquiggleFilter Accuracy Analysis

In [ ]:
# manage imports 
from sklearn import metrics
from itertools import repeat
from numba import njit
from glob import glob
from scipy import stats
import random, h5py, re, os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D  
import multiprocessing as mp
import seaborn as sns
import pandas as pd         
import ont_fast5_api

# set matplotlib params
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

In [ ]:
# from pyguppy_client_lib.pyclient import PyGuppyClient
# from pyguppy_client_lib.helper_functions import package_read, basecall_with_pyguppy
# import mappy

In [ ]:
# user-defined datasets
virus = "lambda"
#virus = "covid"
other = "human"

# user-defined filepaths
data_dir = "data"
img_dir = "img"
os.makedirs(img_dir, exist_ok=True)
kmer_model_fn, k = f"{data_dir}/dna_kmer_model.txt", 6 # 6-mer model
ref_fn = f"{data_dir}/{virus}/reference.fasta"
virus_fast5_dir = f"{data_dir}/{virus}/fast5"
other_fast5_dir = f"{data_dir}/{other}/fast5"
results_dir = f"scripts/results/artifact-eval"

# user-defined analysis params
virus_max_reads = 1000
other_max_reads = 1000
prefix_lengths = np.array(range(1000,9001,1000))
nprefixes = len(prefix_lengths)

## Generate Expected Virus Reference Signal

In [ ]:
def get_fasta(fasta_fn):
    ''' Get base sequence from FASTA filename. '''
    with open(fasta_fn, 'r') as fasta:
        return ''.join(fasta.read().split('\n')[1:])

    
def rev_comp(bases):
    ''' Get reverse complement of sequence. '''
    return bases.replace('A','t').replace('T','a')\
        .replace('G','c').replace('C','g').upper()[::-1]


def load_model(kmer_model_fn):
    ''' Load k-mer model file into Python dict. '''
    kmer_model = {}
    with open(kmer_model_fn, 'r') as model_file:
        for line in model_file:
            kmer, current = line.split()
            kmer_model[kmer] = float(current)
    return kmer_model


def discrete_normalize(seq, bits=8, minval=-4, maxval=4):
    ''' Approximate normalization which converts signal to integer of desired precision. '''
    mean = int(np.mean(seq))
    mean_avg_dev = int(np.mean(np.abs(seq - mean)))
    norm_seq = (seq - mean) / mean_avg_dev
    norm_seq[norm_seq < minval] = minval # threshold
    norm_seq[norm_seq > maxval] = maxval 
    norm_seq = ( (norm_seq - minval) * (2**(bits)/(maxval-minval)) ).astype(int)
    return norm_seq


def ref_signal(fasta, kmer_model):
    ''' Convert reference FASTA to expected reference signal (approximate z-scores). '''
    signal = np.zeros(len(fasta))
    for kmer_start in range(len(fasta)-k):
        signal[kmer_start] = kmer_model[fasta[kmer_start:kmer_start+k]]
    return discrete_normalize(signal*100) # increase dist between floats before rounding

In [ ]:
# create COVID reference using (z-score normalized) 
# expected k-mer currents for forward/backward reference FASTA
ref_fasta = get_fasta(ref_fn)
kmer_model = load_model(kmer_model_fn)
fwd_ref_sig = ref_signal(ref_fasta, kmer_model)
rev_ref_sig = ref_signal(rev_comp(ref_fasta), kmer_model)
ref = np.concatenate((fwd_ref_sig, rev_ref_sig))

## Read Preparation

In [ ]:
def get_stall_end(signal, stall_threshold=3, 
                  stall_events=2, stall_event_len=3):
    ''' Determine the end of the DNA stall region. '''
    
    # take average of a few samples to reduce variation
    events = []
    for event in range(0, len(signal), stall_event_len):
        events.append(np.mean(signal[event:event+stall_event_len]))
    
    # find where we exceed threshold for a few consecutive events
    above_threshold_count = 0
    event_pos = 0
    for event in events:
        event_pos += 1
        if event > stall_threshold:
            above_threshold_count += 1
        else:
            above_threshold_count = 0
        if above_threshold_count == stall_events:
            break
            
    # find where we go back below threshold
    below_threshold_count = 0
    for event in events[event_pos:]:
        event_pos += 1
        if event < stall_threshold:
            below_threshold_count += 1
        else:
            below_threshold_count = 0
        if below_threshold_count == stall_events:
            break
            
    return event_pos * stall_event_len


def trim(signal):
    ''' Trims signal by detecting stall (and eventually adapter). '''
    stall_end = get_stall_end(stats.zscore(signal))
    return signal[stall_end+1000 : stall_end+1000+max(prefix_lengths)], stall_end


def filter_outliers(signal, minval=-4, maxval=4):
    ''' Force FAST5 signal values into expected range. '''
    
    # return empty signals as-is
    if not len(signal): return signal
    
    # upper threshold
    for idx, x in enumerate(signal):
        if x > maxval:
            # other values above max -> threshold to max
            if (idx+1 < len(signal) and signal[idx+1] > 3) or \
            (idx > 0 and signal[idx-1] > maxval):
                signal[idx] = maxval
            # otherwise, single outlier -> interpolate
            elif idx == 0:
                signal[idx] = signal[1]
            elif idx+1 == len(signal):
                signal[idx] = signal[idx-1]
            else:
                signal[idx] = (signal[idx-1] + signal[idx+1]) / 2
                
    # lower threshold
    for idx, x in enumerate(signal):
        if x < minval:
            # other values below min -> threshold to min
            if (idx+1 < len(signal) and signal[idx+1] < -3) or \
            (idx > 0 and signal[idx-1] < minval):
                signal[idx] = minval
            # otherwise, single outlier -> interpolate
            elif idx == 0:
                signal[idx] = signal[1]
            elif idx+1 == len(signal):
                signal[idx] = signal[idx-1]
            else:
                signal[idx] = (signal[idx-1] + signal[idx+1]) / 2
                
    return signal


class Read():
    ''' Store FAST5 read data. '''
    def __init__(self, signal, read_id, offset=0, scaling=1.0):                                                                                                                                                                               
        self.signal = signal                                                                                                                                                                                                                  
        self.read_id = read_id                                                                                                                                                                                                                
        self.total_samples = len(signal)                                                                                                                                                                                                      
        self.daq_offset = offset                                                                                                                                                                                                              
        self.daq_scaling = scaling                                                                                                                                                                                                            
        self.read_tag = random.randint(0, int(2**32 - 1))  

        
def ba_preprocess_read(uuid, length):
    ''' Extract read data from FAST5 file for basecalling. '''
    readname = f"read_{uuid}"
    fast5_file = h5py.File(full_index[uuid], 'r')
    signal = np.array(fast5_file[readname]['Raw']['Signal'][:], dtype=np.int16)
    signal, trimmed = trim(signal)
    if len(signal) < max(prefix_lengths): return None
    signal_dig = fast5_file[readname]['channel_id'].attrs['digitisation']
    signal_offset = fast5_file[readname]['channel_id'].attrs['offset']
    signal_range = fast5_file[readname]['channel_id'].attrs['range']
    signal_scaling = signal_range / signal_dig
    return Read(signal, readname, offset=signal_offset, scaling=signal_scaling)

    
def preprocess_read(uuid):
    ''' Return preprocessed read from specified FAST5 file. '''
    readname = f"read_{uuid}"
    fast5_file = h5py.File(full_index[uuid], 'r')
    signal = np.array(fast5_file[readname]['Raw']['Signal'][:], dtype=np.int16)
    length = signal.shape[0]
    signal, trimmed = trim(signal)
    if len(signal) < max(prefix_lengths): return None
    new_signal = np.array(signal, dtype=float)
    for start in range(0, len(signal), 500):
        new_signal[start:start+500] = \
            discrete_normalize(signal[:start+500])[start:start+500]
    return new_signal, trimmed, length


def get_index(index_filename):
    ''' Read index data structure from file. '''
    index_file = open(index_filename, 'r')
    index = {}
    for line in index_file:
        uuid, fname = re.split(r'\t+', line)
        index[uuid] = fname.rstrip()
    index_file.close()
    return index


def create_index(fast5_dir, force=False):
    ''' Create file which stores read FAST5 to UUID mappings. '''

    # return existing index if possible
    index_fn = f'{fast5_dir}/index.db'
    if not force and os.path.exists(index_fn):
        return get_index(index_fn)

    # remove existing index
    if os.path.exists(index_fn):
        os.remove(index_fn)

    # create new index    
    index_file = open(index_fn, 'w')

    # iterate through all FAST5 files in directory
    for subdir, dirs, files in os.walk(fast5_dir):
        for filename in files:
            ext = os.path.splitext(filename)[-1].lower()
            if ext == ".fast5":

                # print read uuid and filename to index
                fast5_file = h5py.File(os.path.join(subdir, filename), 'r')
                if 'Raw' in fast5_file: # single-FAST5
                    for readname in fast5_file['Raw']['Reads']:
                        uuid = fast5_file['Raw']['Reads'][readname].attrs['read_id']
                        print('{}\t{}'.format(uuid.decode('utf-8'), \
                                os.path.join(subdir, filename)), file=index_file)
                else: # multi-FAST5
                    for readname in fast5_file:
                        uuid = readname[5:] # remove 'read_' naming prefix
                        print('{}\t{}'.format(uuid, \
                                os.path.join(subdir, filename)), file=index_file)

    # cleanup and return results
    index_file.close()
    return get_index(index_fn)

In [ ]:
# create read UUID -> FAST5 filename mapping
virus_index = create_index(virus_fast5_dir)
other_index = create_index(other_fast5_dir)
full_index = {**virus_index, **other_index}

In [ ]:
# select random subset of reads
if virus == "covid":
    # COVID reads were reverse-transcribed RNA->DNA, so most are short.
    # To ensure we have enough long reads for evaluating accuracy at longer sDTW read lengths, use a lot.
    random.seed(42)
    virus_readnames = random.choices(list(virus_index.keys()), k=virus_max_reads*1000)
else: 
    random.seed(7)
    virus_readnames = random.choices(list(virus_index.keys()), k=virus_max_reads*2)
other_readnames = random.choices(list(other_index.keys()), k=other_max_reads*2)

## Baseline: Basecalling and Alignment Read Until
Running baseline adds lots of complicated dependencies, and isn't that useful to verify. Commenting for now.

In [ ]:
# # initialize minimap2 aligner
# aligner = mappy.Aligner(
#     fn_idx_in = ref_fn,
#     preset = "map-ont",
#     best_n = 1,
#     k = 15
# )

# # initialize guppy-lite basecall server
# basecaller = PyGuppyClient(
#     address = "127.0.0.1:1234", 
#     config = "dna_r9.4.1_450bps_fast.cfg",
#     server_file_load_timeout=10
# )
# basecaller.connect()

In [ ]:
# def basecall(packets):
#     calls = []                                                                                                                                                                                                                            
#     sent, rcvd = 0, 0                                                                                                                                                                                                                         
#     while sent < len(packets):                                                                                                                                                                                                            
#         success = basecaller.pass_read(packets[sent])                                                                                                                                                                                     
#         if not success:                                                                                                                                                                                                                       
#             print('ERROR: Failed to basecall read.')                                                                                                                                                                                          
#             break                                                                                                                                                                                                                             
#         else:                                                                                                                                                                                                                                 
#             sent += 1                                                                                                                                                                                                                         
#     while rcvd < len(packets):                                                                                                                                                                                                            
#         result = basecaller.get_completed_reads()                                                                                                                                                                                             
#         rcvd += len(result)                                                                                                                                                                                                                   
#         calls.extend(result)
#     return calls

In [ ]:
# ba_virus_scores = np.zeros((nprefixes, virus_max_reads))
# ba_other_scores = np.zeros((nprefixes, other_max_reads))
# with mp.Pool() as pool:
#     for prefix_idx, length in enumerate(prefix_lengths):
        
#         # trim reads
#         ba_virus_reads = list(filter(None, pool.starmap(
#                     ba_preprocess_read, zip(virus_readnames, repeat(length)))))[:virus_max_reads]
#         ba_other_reads = list(filter(None, pool.starmap(
#                     ba_preprocess_read, zip(other_readnames, repeat(length)))))[:other_max_reads]
                           
#         # package read data
#         virus_pkts = [package_read(
#             read_tag = read.read_tag, 
#             read_id = read.read_id, 
#             raw_data = read.signal, 
#             daq_offset = float(read.daq_offset), 
#             daq_scaling = float(read.daq_scaling)
#         ) for read in ba_virus_reads]
#         other_pkts = [package_read(
#             read_tag = read.read_tag, 
#             read_id = read.read_id, 
#             raw_data = read.signal, 
#             daq_offset = float(read.daq_offset), 
#             daq_scaling = float(read.daq_scaling)
#         ) for read in ba_other_reads]

#         # basecall
#         virus_calls = basecall(virus_pkts)
#         other_calls = basecall(other_pkts)
        
#         # align
#         for call_idx, call in enumerate(virus_calls):
#             try:
#                 alignment = next(aligner.map(call['datasets']['sequence']))
#                 ba_virus_scores[prefix_idx, call_idx] = alignment.mapq
#             except(StopIteration):
#                 pass # no alignment
#         for call_idx, call in enumerate(other_calls):
#             try:
#                 alignment = next(aligner.map(call['datasets']['sequence']))
#                 ba_other_scores[prefix_idx, call_idx] = alignment.mapq
#             except(StopIteration):
#                 pass # no alignment

## Hardware Algorithm: sDTW Alignment Read Until

In [ ]:
# trim all reads
with mp.Pool() as pool:
    virus_reads, virus_trims, virus_lengths = \
        list(map(list, zip(*filter(None, pool.map(preprocess_read, virus_readnames)))))
    other_reads, other_trims, other_lengths = \
        list(map(list, zip(*filter(None, pool.map(preprocess_read, other_readnames)))))

In [ ]:
# warn user if not enough long reads for accuracy analyses
if len(virus_reads) < virus_max_reads:
    print(f'ERROR: only {len(virus_reads)} virus reads long enough, requested {virus_max_reads}')
if len(other_reads) < other_max_reads:
    print(f'ERROR: only {len(other_reads)} other reads long enough, requested {other_max_reads}')
    
# keep only 'max_reads' for further analysis
virus_reads, virus_trims, virus_lengths = virus_reads[:virus_max_reads], \
    virus_trims[:virus_max_reads], virus_lengths[:virus_max_reads]
other_reads, other_trims, other_lengths = other_reads[:other_max_reads], \
    other_trims[:other_max_reads], other_lengths[:other_max_reads]

In [ ]:
@njit()
def sdtw(seq):
    ''' Returns minimum alignment score for subsequence DTW. '''
    
    # initialize column vectors
    prev_consec = np.zeros((len(seq)))
    curr_consec = np.zeros((len(seq)))
    prev_cost = np.zeros((len(seq)))
    curr_cost = np.zeros((len(seq)))
    min_cost = np.zeros((len(seq)))
    prev_cost[0] = abs(seq[0]-ref[0])
    min_cost[0] = abs(seq[0]-ref[0])
    for i in range(1, len(seq)):
        prev_cost[i] = prev_cost[i-1] + abs(seq[i]-ref[0])
        min_cost[i] = min_cost[i-1] + abs(seq[i]-ref[0])
    
    # compute entire cost matrix
    for j in range(1, len(ref)):
        bonus = 10
        for i in range(1, len(seq)):
            move = prev_cost[i-1] - prev_consec[i-1]*bonus < curr_cost[i-1]
            if move:
                curr_consec[i] = 0
                curr_cost[i] = prev_cost[i-1] - prev_consec[i-1]*bonus + abs(seq[i]-ref[j])
            else:
                curr_consec[i] = min(10, prev_consec[i] + 1)
                curr_cost[i] = curr_cost[i-1] + abs(seq[i]-ref[j])
            min_cost[i] = min(min_cost[i], curr_cost[i])
        for i in range(len(seq)):
            prev_consec[i] = curr_consec[i]
            curr_consec[i] = 0
            prev_cost[i] = curr_cost[i]
            curr_cost[i] = 0
    
    # return cost of optimal alignment
    cost_mins = np.zeros((len(prefix_lengths),))
    for i, l in enumerate(prefix_lengths):
        if l <= len(seq):
            cost_mins[i] = min_cost[l]
    return cost_mins

In [ ]:
with mp.Pool() as pool:
    print(f'Aligning {virus} reads...', flush=True)
    virus_scores_list = pool.map(sdtw, virus_reads)
    print(f'Aligning {other} reads...', flush=True)
    other_scores_list = pool.map(sdtw, other_reads)

## Save Results

In [ ]:
# move data to numpy array for easy sorting/calculations
virus_scores = np.zeros((nprefixes, len(virus_scores_list)))
for idx, scores in enumerate(virus_scores_list):
    for i in range(nprefixes):
        virus_scores[i,idx] = scores[i]
other_scores = np.zeros((nprefixes, len(other_scores_list)))
for idx, scores in enumerate(other_scores_list):
    for i in range(nprefixes):
        other_scores[i,idx] = scores[i]
        
# save results
os.makedirs(results_dir, exist_ok=True)
np.save(f"{results_dir}/prefix_lengths", prefix_lengths)
np.save(f"{results_dir}/{virus}_trims", virus_trims)
np.save(f"{results_dir}/{virus}_lengths", virus_lengths)
np.save(f"{results_dir}/{virus}_scores", virus_scores)
np.save(f"{results_dir}/{other}-{virus}_trims", other_trims)
np.save(f"{results_dir}/{other}-{virus}_lengths", other_lengths)
np.save(f"{results_dir}/{other}-{virus}_scores", other_scores)

In [ ]:
# # save basecall-align results as well
# np.save(f"{results_dir}/ba_{virus}_scores", ba_virus_scores)
# np.save(f"{results_dir}/ba_{other}-{virus}_scores", ba_other_scores)

In [ ]:
# load cached results (allows skipping data generation above)
prefix_lengths = np.load(f"{results_dir}/prefix_lengths.npy")
virus_trims = np.load(f"{results_dir}/{virus}_trims.npy")
virus_lengths = np.load(f"{results_dir}/{virus}_lengths.npy")
virus_scores = np.load(f"{results_dir}/{virus}_scores.npy")
other_trims = np.load(f"{results_dir}/{other}-{virus}_trims.npy")
other_lengths = np.load(f"{results_dir}/{other}-{virus}_lengths.npy")
other_scores = np.load(f"{results_dir}/{other}-{virus}_scores.npy")

In [ ]:
# load cached basecall-align results as well
ba_virus_scores = np.load(f"{results_dir}/ba_{virus}_scores.npy")
ba_other_scores = np.load(f"{results_dir}/ba_{other}-{virus}_scores.npy")

In [ ]:
def get_stats(virus_scores, other_scores, thresh):
    ''' Return F-scores (assumes sorted input). '''
    fscores = np.zeros(nprefixes)
    precs = np.zeros(nprefixes)
    recalls = np.zeros(nprefixes)
    for i in range(nprefixes):
        # short reads don't receive a score, so ignore in accuracy metrics
        long_virus = np.count_nonzero(virus_scores[i])
        short_virus = virus_scores.shape[1]-long_virus
        tp = np.searchsorted(virus_scores[i], thresh) - short_virus
        fn = long_virus - tp
        long_other = np.count_nonzero(other_scores[i])
        short_other = other_scores.shape[1]-long_other
        fp = np.searchsorted(other_scores[i], thresh) - short_other
        precs[i] = 0 if not tp+fp else tp / (tp+fp)
        recalls[i] = 0 if not tp+fn else tp / (tp+fn)  
        fscores[i] = 0 if not tp+fp+fn else tp / (tp + 0.5*(fp + fn))
    return fscores, precs, recalls

In [ ]:
# sort arrays (for fast f-score calculation)
virus_scores = np.sort(virus_scores)
other_scores = np.sort(other_scores)
min_score = min(np.min(virus_scores), np.min(other_scores))
max_score = max(np.max(virus_scores), np.max(other_scores))

# calculate all f-scores, and save the best thresholds
best_threshs = np.zeros(nprefixes)
best_fscores = np.zeros(nprefixes)
best_precs = np.zeros(nprefixes)
best_recalls = np.zeros(nprefixes)
for thresh in np.linspace(min_score, max_score, num=100):
    fscores, precs, recalls = get_stats(virus_scores, other_scores, thresh)
    for i in range(nprefixes):
        if fscores[i] > best_fscores[i]:
            best_fscores[i] = fscores[i]
            best_precs[i] = precs[i]
            best_recalls[i] = recalls[i]
            best_threshs[i] = thresh + 0.01
            
# save accuracy results
np.save(f"{results_dir}/{other}-{virus}_fscores", best_fscores)
np.save(f"{results_dir}/{other}-{virus}_precisions", best_precs)
np.save(f"{results_dir}/{other}-{virus}_recalls", best_recalls)

### Accuracy Analysis (Figure 17a)

In [ ]:
dtw_lengths = [1000, 2000, 3000, 5000]
ba_lengths = [1000]

# initialize plots
mpl.rcParams.update({'font.size': 14})
dtw_indices = [np.where(prefix_lengths == t)[0] for t in dtw_lengths]
ba_indices = [np.where(prefix_lengths == t)[0] for t in ba_lengths]
fig, ax = plt.subplots()

# all DTW plots
linestyles=['-', '--', '-.', ':', '-']
dashes = [[10,0], [4,2], [6,1.5,1,1.5], [8,4], [8,2,2,2,2,2]]
colors = ['C0', 'C1', 'C2', 'C4', 'C5']
j = 0
for i, l in zip(dtw_indices, dtw_lengths):
    i = int(i)
    minval = min(np.min(virus_scores[i]), np.min(other_scores[i]))-1
    maxval = max(np.max(virus_scores[i]), np.max(other_scores[i]))+1
    thresholds = np.linspace(minval, maxval, num=100)

    other_discard_rate, virus_discard_rate = [], []
    for t in thresholds:
        virus_discard_rate.append(np.sum(virus_scores[i] > t) / len(virus_scores[i]))
        other_discard_rate.append(np.sum(other_scores[i] > t) / len(other_scores[i]))
    ax.plot(virus_discard_rate, other_discard_rate, linestyle=linestyles[j], linewidth=3, dashes=dashes[j], color=colors[j])
    j+=1
    
# # all Guppy + Minimap2 plots
# for i, l in zip(ba_indices, ba_lengths):
#     i = int(i)
#     minval = min(np.min(ba_virus_scores[i]), np.min(ba_other_scores[i]))-1
#     maxval = max(np.max(ba_virus_scores[i]), np.max(ba_other_scores[i]))+1
#     thresholds = np.linspace(minval, maxval, num=100)

#     other_discard_rate, virus_discard_rate = [], []
#     for t in thresholds:
#         virus_discard_rate.append(sum(ba_virus_scores[i] < t) / len(ba_virus_scores[i]))
#         other_discard_rate.append(sum(ba_other_scores[i] < t) / len(ba_other_scores[i]))
#     ax.plot(virus_discard_rate, other_discard_rate, linestyle=':', linewidth=3, color='C3')
    
ax.set_xlabel('Lambda Phage Discard Rate', fontsize=18)
ax.set_ylabel('Human Discard Rate', fontsize=18)
ax.set_xlim((-0.1, 1.1))
ax.set_ylim((-0.1, 1.1))
ax.set_title('Accuracy')
plt.savefig(f"{img_dir}/fig17_discard-rates.png")
plt.show()

## Read Until Runtime
Analyze runtime as a function of accuracy and multi-stage thresholding

In [ ]:
class Reads():
    def __init__(self, results_dir, virus_acc, other_acc, virus_eval=None):
        if not virus_eval:
            virus_eval = virus_acc
        
        self.prop_virus = 0.01        # proportion virus
        self.prop_other = 1 - self.prop_virus
        self.prefix_lengths = np.load(f'{results_dir}/prefix_lengths.npy')

        self.ba_virus_scores = np.load(f'{results_dir}/ba_{virus_acc}_scores.npy')
        self.virus_scores = np.load(f'{results_dir}/{virus_acc}_scores.npy')
        self.virus_lengths = np.load(f'{results_dir}/{virus_eval}_lengths.npy')
        if virus_eval == "covid":
            # we've subselected reads long enough to do our accuracy analysis,
            # but actual average read length of reverse transcribed DNA is much lower
            self.avg_virus_length = 11814 # calculated from dataset
            self.target_genome_size = 30_000.0 # bases
        else:
            self.avg_virus_length = np.mean(self.virus_lengths)
            self.target_genome_size = 100_000.0 # bases
        self.virus_trims = np.load(f'{results_dir}/{virus_eval}_trims.npy')
        self.avg_virus_trim = np.mean(self.virus_trims)

        self.ba_other_scores = np.load(f'{results_dir}/ba_{other}-{virus_acc}_scores.npy')
        self.other_scores = np.load(f"{results_dir}/{other}-{virus_acc}_scores.npy")
        self.other_lengths = np.load(f"{results_dir}/{other}-{virus_eval}_lengths.npy")
        if virus_eval == "covid":
            # we use a normal human DNA dataset, but take average read length from
            # a reverse-transcribed DNA dataset here to match lab prep for COVID
            self.avg_other_length = 12551 # calculated from rtDNA dataset
        else:
            self.avg_other_length = np.mean(self.other_lengths)
        self.other_trims = np.load(f"{results_dir}/{other}-{virus_eval}_trims.npy")
        self.avg_other_trim = np.mean(self.other_trims)
    
    
class Flowcell():
    def __init__(self, channels = 512):
        self.chemistry = 'r9.4.1'
        self.sampling_rate = 4000     # samples/sec, known
        self.minknow_latency = 0.070  # sec, measured
        self.channels = channels
        
        
class Classifier():
    def __init__(self, method='sf', watts=1):
        self.method = method
        if method == 'sf': # SquiggleFilter measured
            self.throughput = 23_364 * 2000 * watts
            self.latency = 0.0000408
        elif method == 'ba': # BasecallAlign measured
            self.throughput = 550 * 2000
            self.latency = 0.149
        else:
            raise Exception("Unknown Read Until classifier type.")
    

class Run():
    def __init__(self, reads, clf='sf', flowcell=Flowcell(), watts=1):
        self.flowcell = flowcell
        self.clf = Classifier(clf, watts)
        self.reads = reads
        self.target_coverage = 30.0
        self.coverage_bias = 10.0     # estimated
        self.fwd_tr = 400.0           # bases / sec
        self.rev_tr = 100_000.0       # bases / sec
        self.capture_time = 1.0       # sec
        self.sr = self.flowcell.sampling_rate
    
    
    def get_simple_runtime(self):
        max_throughput = self.flowcell.channels * self.sr
        virus_time = self.reads.prop_virus * \
            (self.capture_time + self.reads.avg_virus_length/self.sr)
        other_time = self.reads.prop_other * \
            (self.capture_time + self.reads.avg_other_length/self.sr)
        useful_time = self.reads.prop_virus * \
            (self.reads.avg_virus_length-self.reads.avg_virus_trim) / self.sr
        useful_throughput = max_throughput * useful_time / (virus_time + other_time)
        duration = self.reads.target_genome_size * (self.sr/self.fwd_tr) * \
            self.target_coverage * self.coverage_bias / useful_throughput
        return duration
    
    
    def get_read_until_runtime(self, prefix_indices, thresholds):
        
        # get runtime without read until
        max_throughput = self.flowcell.channels * self.sr
        simple_virus_time = self.reads.prop_virus * \
            (self.capture_time + self.reads.avg_virus_length/self.sr)
        simple_other_time = self.reads.prop_other * \
            (self.capture_time + self.reads.avg_other_length/self.sr)
        simple_useful_time = self.reads.prop_virus * \
            (self.reads.avg_virus_length-self.reads.avg_virus_trim) / self.sr
        simple_prop_useful_time = simple_useful_time / \
            (simple_virus_time + simple_other_time)
        
        # determine percentage of pores that can perform Read Until
        # - estimate required basecall throughput from simple sequencing
        bc_time = self.reads.prop_virus * self.reads.avg_virus_length/self.sr + \
            self.reads.prop_other * self.reads.avg_other_length/self.sr
        bc_throughput = max_throughput * (bc_time/ (simple_virus_time + simple_other_time))
        prop_ru = min(1.0, self.clf.throughput/bc_throughput)
        prop_simple = 1 - prop_ru
        
        # calculate sequencing runtime for multiple thresholds
        if self.clf.method == 'sf':
            rem_virus_scores = self.reads.virus_scores.copy()
            rem_other_scores = self.reads.other_scores.copy()
        else:
            rem_virus_scores = -self.reads.ba_virus_scores.copy()
            rem_other_scores = -self.reads.ba_other_scores.copy()   
            
        eject_virus_time, eject_other_time = 0, 0
        for i, thresh in zip(prefix_indices, thresholds):
            
            # device continues sequencing as we make a read-until decision
            length = self.reads.prefix_lengths[i]
            samples = length + self.sr * \
                (self.clf.latency + self.flowcell.minknow_latency)
            reversal_latency = samples / (self.rev_tr * (self.sr/self.fwd_tr))
            latency = self.clf.latency + self.flowcell.minknow_latency + reversal_latency
            
            # choose which reads to keep, count those ejected
            keep_virus = rem_virus_scores[i] < thresh
            rem_virus_scores = rem_virus_scores[:,keep_virus]
            n_eject_virus = len(keep_virus) - sum(keep_virus)
            keep_other = rem_other_scores[i] < thresh
            rem_other_scores = rem_other_scores[:,keep_other]
            n_eject_other = len(keep_other) - sum(keep_other)
            
            # update time spent sequencing ejected reads
            eject_virus_time += n_eject_virus * self.reads.prop_virus * \
                (self.capture_time + length/self.sr + latency)
            eject_other_time += n_eject_other * self.reads.prop_other * \
                (self.capture_time + length/self.sr + latency)
            
        # update total time spent sequencing each type of read
        ru_useful_time = len(rem_virus_scores[0]) * self.reads.prop_virus * \
            (self.reads.avg_virus_length - self.reads.avg_virus_trim) / self.sr
        ru_virus_time = eject_virus_time + len(rem_virus_scores[0]) * self.reads.prop_virus * \
            (self.capture_time + self.reads.avg_virus_length / self.sr)
        ru_other_time = eject_other_time + len(rem_other_scores[0]) * self.reads.prop_other * \
            (self.capture_time + self.reads.avg_other_length / self.sr)
        ru_prop_useful_time = ru_useful_time / (ru_virus_time + ru_other_time)
        
        # calculate duration based on simple/read until split
        prop_useful_time = prop_ru * ru_prop_useful_time + \
            prop_simple * simple_prop_useful_time
        useful_throughput = prop_useful_time * max_throughput + 0.0001
        duration = self.reads.target_genome_size * (self.sr/self.fwd_tr) * \
            self.target_coverage * self.coverage_bias / useful_throughput
        return duration

### Read Until Runtime (Figure 17b and 17c)

Use the filter accuracy metrics from lambda phage and the known read length distribution from COVID to estimate the optimal read until thresholds for COVID. To do this, cached statistics must have been generated for both 'covid' and 'lambda'. First run all cells with `virus='lambda'` (in Cell 3), then do the same with `virus='covid'`.

In [ ]:
plot_indices = [0,1,2,4,7]
plot_prefixes = [1000,2000,3000,5000,8000]
best_threshs = [0]*nprefixes

if os.path.isfile(f'{results_dir}/lambda_scores.npy') and os.path.isfile(f'{results_dir}/covid_scores.npy'):
    read_data = Reads(results_dir, virus_acc='lambda', other_acc=other, virus_eval='covid')
    run = Run(read_data)
    for i1, p1 in zip(plot_indices, plot_prefixes):
        max_score = max(np.max(virus_scores[i1]), np.max(other_scores[i1]))
        min_score = max(1, min(np.min(virus_scores[i1]), np.min(other_scores[i1])))
        times = []
        thresholds = np.linspace(min_score, max_score, 100)
        for t in thresholds:
            times.append(run.get_read_until_runtime([i1], [t]))

        best = np.argmin(times)
        best_threshs[i1] = thresholds[best]
else:
    print(f"You must run data analysis on covid and lambda before determining optimal thresholds.")

In [ ]:
for v in ['lambda', 'covid']:
    if os.path.isfile(f'{results_dir}/{v}_scores.npy'):
        read_data = Reads(results_dir, v, other)
        run = Run(read_data)
        xmax = 80000
        plt.plot(np.linspace(0,xmax,20), [run.get_simple_runtime()]*20, color='k', linestyle=':')

        linestyles=['-', '--', '-.', ':', '-']
        dashes = [[10,0], [4,2], [6,1.5,1,1.5], [8,4], [8,2,2,2,2,2]]
        colors = ['C0', 'C1', 'C2', 'C4', 'C5']
        j = 0
        first=True
        plot_indices = [0,1,2,4,7]
        plot_prefixes = [1000,2000,3000,5000,8000]

        for i1, p1 in zip(plot_indices, plot_prefixes):
            max_score = max(np.max(virus_scores[i1]), np.max(other_scores[i1]))
            min_score = max(1, min(np.min(virus_scores[i1]), np.min(other_scores[i1])))
            times = []
            thresholds = np.linspace(min_score, max_score, 100)
            for t in thresholds:
                times.append(run.get_read_until_runtime([i1], [t]))

            if v == "lambda":
                best = np.argmin(times)
                if first:
                    first=False
                    plt.plot(thresholds[best], times[best], marker='*', markersize=9, color='k', linestyle='', zorder=2)
                else:
                    plt.plot(thresholds[best], times[best], marker='*', markersize=9, color='k', linestyle='', label='_nolegend_', zorder=2)
            else:
                if first:
                    best = best_threshs[i1]
                    first=False
                    time = np.interp(best, thresholds, times)
                    plt.plot(best, time, marker='*', markersize=9, color='k', linestyle='', zorder=2)
                else:
                    best = best_threshs[i1]
                    time = np.interp(best, thresholds, times)
                    plt.plot(best, time, marker='*', markersize=9, color='k', linestyle='', label='_nolegend_', zorder=2)

            plt.plot(thresholds, times, dashes=dashes[j], color=colors[j], linewidth=3, zorder=1)
            plt.ylabel('Read Until runtime (s)', fontsize=18)
            plt.xlabel('sDTW Alignment Score Threshold', fontsize=18)
            j+= 1

        plt.xlim(0,xmax)
        plt.ylim(3000, run.get_simple_runtime()*1.3)
        plt.yscale('log')
        if v == 'lambda':
            plt.title('Lambda Phage')
        else:
            plt.title('SARS-CoV-2')
            plt.plot([], [], linestyle=':', linewidth=3, color='C3')
            leg = plt.legend(['No Read Until', 'Selected Threshold'] + [f'SquiggleFilter {l} samples' for l in plot_prefixes] + ['Guppy-lite 1000 samples'], 
                             loc=(1.04,0.1), handlelength=4)
            leg.legendHandles[2]._legmarker.set_markersize(9)
        plt.savefig(f"{img_dir}/fig17_{virus}-read-until-time.png")
        plt.show()
    else:
        print(f"You must run data analysis on '{v}' before generating corresponding plot.")

### Score Distribution (Figure 11)

In [ ]:
plot_prefixes = [1000, 2000, 3000, 5000, 8000]
fig, ax = plt.subplots(1,len(plot_prefixes), figsize=(12,2))
j = 0
for i, l in enumerate(prefix_lengths):
    if l in plot_prefixes:
        minval = min(np.min(virus_scores[i]), np.min(other_scores[i]))
        maxval = max(np.max(virus_scores[i]), np.max(other_scores[i]))
        ax[j].hist(virus_scores[i], bins=np.linspace(minval, maxval, num=100), facecolor='#1a4099', alpha=0.7)
        ax[j].hist(other_scores[i], bins=np.linspace(minval, maxval, num=100), facecolor='#ebc100', alpha=0.5)
        ax[j].axvline(best_threshs[i], color='k', linestyle='--')
        ax[j].set_title('{} Samples'.format(l), fontsize=16)
        ax[j].set_ylim((0,130))
        ax[j].set_xlim(0, best_threshs[i]*2)
        j += 1

    ax[0].legend(['threshold', virus, other], loc='upper left', fontsize=15)
    ax[2].set_xlabel('sDTW Alignment Cost', fontsize=20)
    ax[1].set_yticklabels([])
    ax[2].set_yticklabels([])
    ax[3].set_yticklabels([])
    ax[4].set_yticklabels([])
    ax[0].set_ylabel('Read Count', fontsize=20)

plt.savefig(f"{img_dir}/fig11_cost-distribution.png")
plt.show()

## Other Figures

### Progression of SARS-CoV-2 Testing (Figure 2)

In [ ]:
mpl.rcParams.update({'font.size': 10})   
df = pd.read_csv(f'{data_dir}/owid-covid-data.csv', header=None) 
df[1] = df[1].diff()                                                             
                                                                                 
ax = df.plot(color="#6eaaef", figsize=(4.5,2), zorder=1)                         
ax.set_yticks([100_000, 500_000, 1_000_000, 2_000_000])                          
ax.set_yticklabels(["100,000", "500,000", "1 million", "2 million"])             
ax.set_ylim(0, 2_500_000)                                                        
                                                                                 
ax.set_xticks([0, 196, 366])                                                     
ax.set_xticklabels(["Jan 1 2020", "July 15", "Jan 1 2021"])                      
ax.legend().remove()                                                             
                                                                                
plt.vlines(x=13, linestyle="--", ymin=0, ymax=2_500_000, color="k")              
plt.text(14, 1_400_000, 'SARS-CoV-2\nSequenced')                                 
                                                                                 
plt.arrow(14, 200_000, 181, 0, fc='k', ec='k', zorder=2)                         
plt.plot(18, 200_000, marker='<', color='k')                                     
plt.plot(189, 200_000, marker='>', color='k')                                    
plt.text(23, 260_000, 'Insufficient Testing')                                    
                                                                                 
plt.vlines(x=196, linestyle='--', ymin=0, ymax=2_500_000, color='k')             
plt.text(197, 1_400_000, '1 Million\nDaily Tests')                               
                                                                                 
lines = [Line2D([0], [0], color="#6eaaef", lw=3)]                                
ax.legend(lines, ["Daily US COVID-19 Tests"], loc='upper right')                 
                                                                               
plt.tight_layout()                                                               
plt.savefig(f"{img_dir}/fig2_covid-tests.png")
plt.show()

### Pipeline Runtime Proportions (Figure 5)

In [ ]:
mpl.rcParams.update({'font.size': 14})                                    
                                                                                 
fig, axes = plt.subplots(1,2)                                                    
tools = ['Basecalling', 'Alignment', 'Variant Calling']                          
covid_prop100 = [87.72, 2.75, 9.53]                                              
covid_prop1000 = [95.95, 3.01, 1.04]                                             
colors = ["#1a4099", "#ebc100", "#9c9c9c"]                                       
labels0 = [f'{tool} ({pct}%)' for tool, pct in zip(tools, covid_prop100)]        
labels1 = [f'{tool} ({pct}%)' for tool, pct in zip(tools, covid_prop1000)]          
                                                                                 
patches0, _ = axes[0].pie(covid_prop100, startangle=90, colors=colors)           
patches1, _ = axes[1].pie(covid_prop1000, startangle=90, colors=colors)          
axes[0].legend(patches0, labels0, loc=(0,-0.3))                                  
axes[1].legend(patches1, labels1, loc=(0,-0.3))                                  
plt.tight_layout()                                                               
fig.savefig(f'{img_dir}/fig5_pipeline-runtimes.png') 

### Virus Genome Sizes (Figure 10)

In [ ]:
mpl.rcParams.update({'font.size': 17})                                                                                 
                                                                                 
def add_datapoint(ax, data):                                                     
    name = data[0]                                                               
    size = data[1]                                                               
    na = data[2]                                                                 
    strands = data[3]                                                            
    height = data[4]                                                             
    ax.text(size, height+0.2, name, {'color': "#ebc100" if na == "DNA" else "#1a4099"}, ha="left", va="center")
    ax.plot(size, height, color="#ebc100" if na == "DNA" else "#1a4099", markersize=12, marker="P" if strands=="ds" else ".")
                                                                                 
virus_genomes = [                                                                
    ["Coxsackie", 7200, "RNA", "ss", 0.7],                                       
    ["Polio", 7500, "RNA", "ss", 1.4],                                           
    ["HIV", 9800, "RNA", "ss", 2.1],                                             
    ["Zika", 10000, "RNA", "ss", 2.8],          
    ["Dengue", 11000, "RNA", "ss", 3.5],                                         
    ["Yellow Fever", 11000, "RNA", "ss", 4.2],                                   
    ["Borna Disease", 8900, "RNA", "ss", 4.9],                                   
    ["West Nile", 11500, "RNA", "ss", 5.6],                                      
    ["Influenza A", 13500, "RNA", "ss", 0.7],                                    
    ["Influenza B", 14500, "RNA", "ss", 1.4],                                    
    ["Mumps", 15400, "RNA", "ss", 2.1],                                          
    ["Measles", 15900, "RNA", "ss", 2.8],                                        
    ["Ebola", 19000, "RNA", "ss", 3.5],                                          
    ["Reovirus", 29200, "RNA", "ds", 0.7],                                       
    ["SARS", 29700, "RNA", "ss", 1.4],                                           
    ["Coronavirus", 29900, "RNA", "ss", 2.1],                                    
    ["Herpes Simplex", 152000, "DNA", "ds", 0.7],                                
    ["Smallpox", 186000, "DNA", "ds", 1.4],                                      
]                                                                                
                                                                                 
fig, ax = plt.subplots(figsize=(10,4))                                           
ax.set_ylim(0, 6.5)                                                              
ax.set_yticks([])                                                                
for v in virus_genomes:                                                          
    add_datapoint(ax, v)                                                         
ax.set_xscale('log')                                                             
ax.set_xlim(5000, 500000)                                                        
ax.set_xticks([5_000, 10_000, 50_000, 100_000, 250_000])                         
ax.set_xticklabels(["5,000","10,000","50,000","100,000", "250,000"])             
ax.set_xlabel("Genome length (bases)")                                           
                                                                                 
# legend                                                                         
ax.add_patch(mpl.patches.Rectangle((110_000, 4), 375_000, 2.3, fill=None, alpha=1))          
ax.plot(120000, 5.8, color="#1a4099", marker=".", markersize=15)                 
ax.text(130000, 5.8, "single stranded RNA", color="#1a4099", va="center")        
ax.plot(120000, 5.1, color="#1a4099", marker="P", markersize=15)                 
ax.text(130000, 5.1, "double stranded RNA", color="#1a4099", va="center")        
ax.plot(120000, 4.4, color="#ebc100", marker="P", markersize=15)                 
ax.text(130000, 4.4, "double stranded DNA", color="#ebc100", va="center")        
                                                                                 
# lines                                                                          
for x in np.arange(0, 6.5, 0.4):                                                 
    ax.plot(50_000, x, marker='P', markersize=5, color='k')                      
    ax.plot(100_000, x, marker='.', markersize=5, color='k')                     
                                                                                 
plt.tight_layout()                                                               
plt.savefig(f'{img_dir}/fig10_virus-genomes.png')
plt.show()

### Latency Results (Figure 16a)

In [ ]:
mpl.rcParams.update({'font.size': 18})                                     
fig, ax = plt.subplots()                                                         
                                                                                 
tools = ['Guppy', 'Guppy\nLite', 'Squiggle\nFilter']                             
colors = ['C3', 'C1', 'C2']                                                      
latencies = [ 1.304, 0.149, (107_000) / (2.5 * 10**9) ]                          
                                                                                 
ax.bar(tools, latencies, color=colors)                                           
                                                                                 
base_lines = [1, 10, 100, 500]                                                   
base_latencies = [.0025, .025, .25, 1.25]                                        
for latency, base in zip(base_latencies, base_lines):                            
    plt.hlines(y=latency, xmin=-0.5, xmax=1.5, color='k', linestyle=':', linewidth=3)
    label = " 1 base" if base == 1 else f' {base} bases'                         
    plt.text(1.5, latency, label, ha='left', va='center')                        
                                                                                 
ax.set_ylabel('Latency (s)')                                                     
ax.set_yscale('log')                                                             
plt.tight_layout()                                                               
fig.savefig(f'{img_dir}/fig16_latency.png')
plt.show()

### Throughput Results (Figure 16b)

In [ ]:
mpl.rcParams.update({'font.size': 20})                                         
fig, ax = plt.subplots(figsize=(15,5))                                                
                                                                                      
systems = ['Guppy', 'Guppy\nLite', ' Guppy', ' Guppy\nLite', 'Squiggle\nFilter', ' Squiggle\nFilter']
colors = ['C3', 'C1', 'C3', 'C1', 'C2', 'C2']                                         
                                                                                      
titan_thru_hac = 387                                                                  
titan_thru_fast = 1524                                                                
compute_ratio_hac = 480941/222959                                                     
compute_ratio_fast = 109888/39656                                                     
                                                                                      
throughputs = [                                                                       
        titan_thru_hac/compute_ratio_hac,                                             
        titan_thru_fast/compute_ratio_fast,                                           
        titan_thru_hac,                                                               
        titan_thru_fast,                                                              
        2.5*10**9 / 107_000,                                                          
        5 * 2.5*10**9 / 107_000 ]                                                     
                                                                                      
ax.bar(systems, throughputs, color=colors)                                            
                                                                                      
ax.axhline(y=512*4000/2000, color='k', linestyle='--', linewidth=3)                   
ax.axhline(y=5*512*4000/2000, color='k', linestyle=':', linewidth=3)                  
ax.axhline(y=24*3000*4000/2000, color='k', linestyle='-.', linewidth=3)               
ax.axhline(y=100*512*4000/2000, color='k', linestyle='-', linewidth=3)                
ax.legend(['MinION output', 'GridION output', 'PromethION output', 'Future MinION output'], labelspacing=0.2, loc=(1.04,0.5))
                                                                                      
ax.set_ylabel('Throughput (reads/s)')                                                 
ax.set_yscale('log')                                                                  
ax.set_xticklabels(systems)                                                           
ax.set_xlim(-0.7, 5.7)                                                                
                                                                                      
                                                                                      
ax.text(0.5, 6000, "Jetson\nXavier\n" + r"$350\mathrm{mm}^2$" + "\n30W", ha='center')
ax.text(2.5, 6000, "Titan\nXP\n" + r"$471\mathrm{mm}^2$" + "\n250W", ha='center')
ax.text(4, 6000, "1-Tile\nASIC\n" + r"$3\mathrm{mm}^2$" + "\n3W", ha='center')   
ax.text(5, 6000, "5-Tile\nASIC\n" + r"$13\mathrm{mm}^2$" + "\n14W", ha='center') 
                                                                                      
plt.tight_layout()                                                                    
fig.savefig(f'{img_dir}/fig16_throughput.png', dpi=300)
plt.show()

### Algorithm Modifications (Figure 18)

In [ ]:
results_dirs = ["final_baseline", "final_no_ref_skip", "final_discrete_norm", "final_abs_value", "final_all", "final_all_bonus"]
linestyles=['-', '-.', '--', ':', '-', ':']
markers=['','','','','*','*']
mpl.rcParams.update({'font.size': 16})
fig, axs = plt.subplots(1,1, figsize=(8,4))
for i, d in enumerate(results_dirs):
    fscores = np.load(f"scripts/results/{d}/fscores.npy")
    axs.plot(np.arange(500,5001,500), fscores, marker=markers[i], linestyle=linestyles[i])
axs.set_ylim(0.5,1)
axs.set_xlim(0,7000)
axs.set_xlabel('Samples')
axs.set_ylabel('F-Score')
axs.legend(["Baseline", "No Reference Deletions", "Integer Normalization", "Absolute Difference", "All Efficiency Optimizations", "All Eff Opts + Match Bonus"],)
plt.savefig(f"{img_dir}/fig18_alg-mods.png")
plt.show()

### Mutation-Dependent Accuracy (Figure 19)

In [ ]:
mpl.rcParams.update({'font.size': 18})                                    
                                                                                 
fig, ax = plt.subplots(1, figsize=(8,3))                                         
                                                                                 
n_mut = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000]        
f_sub = [0.901, 0.9, 0.902, 0.9, 0.9, 0.902, 0.899, 0.896, 0.891, 0.878, 0.836, 0.662, 0.209, 0.027]
f_ins = [0.901, 0.903, 0.901, 0.901, 0.9, 0.902, 0.9, 0.9, 0.893, 0.883, 0.877, 0.807, 0.604, 0.093]
f_del = [0.901, 0.901, 0.903, 0.9, 0.9, 0.898, 0.9, 0.897, 0.884, 0.867, 0.806, 0.438, 0.031, 0.021]
                                                                                 
ax.plot(n_mut, f_sub, linewidth=4, linestyle='-', color='r', alpha=0.5)          
ax.plot(n_mut, f_ins, linewidth=4, linestyle='--', color='g', alpha=0.5)         
ax.plot(n_mut, f_del, linewidth=4, linestyle=':', color='b', alpha=0.5)          
ax.set_ylim(0,1)                                                                 
ax.set_xscale('log')                                                             
ax.set_ylabel('F-score')                                                         
ax.set_xlabel('Number of Mutations')                                             
ax.legend(['Substitutions', 'Insertions', 'Deletions'])                          
                                                                                 
plt.tight_layout()                                                               
plt.savefig(f'{img_dir}/fig19_mutation-dependent-accuracy.png', dpi=300)   
plt.show()

### Projected Future SquiggleFilter Read Until Benefits (Figure 21)

In [ ]:
read_data = Reads(results_dir, virus, other)
mpl.rcParams.update({'font.size': 22})

simple_runtimes = []
sf_runtimes = [[], []]
ba_runtimes = []
channels = np.logspace(2, 9, num=30)
for channel_count in channels:
    flowcell = Flowcell(channel_count)
    for i, w in enumerate([1,5]):
        sf_run = Run(read_data, 'sf', flowcell, w)
        sf_runtimes[i].append(sf_run.get_read_until_runtime([1], [12935]))
    ba_run = Run(read_data, 'ba', flowcell)
    simple_runtimes.append(sf_run.get_simple_runtime())
    ba_runtimes.append(ba_run.get_read_until_runtime([0], [-1]))
fig = plt.figure(figsize=(12,4))
ax = fig.add_subplot(1,1,1)
ax.set_yscale('log')
ax.set_xscale('log')
ax.plot(channels, simple_runtimes, color='k', linestyle=':', linewidth=3)
ax.plot(channels, ba_runtimes, linestyle=':', color='C3', linewidth=4)
ax.plot(channels, sf_runtimes[0], linestyle='--', color='C2', linewidth=4, alpha=0.7)
ax.plot(channels, sf_runtimes[1], linestyle='-.', color='C2', linewidth=4, alpha=0.7)
ax.axvline(512, color='k', linestyle='--', linewidth=3)
ax.axvline(5*512, color='k', linestyle='--', alpha=0.5, linewidth=3)
ax.axvline(3000*24, color='k', linestyle='--', alpha=0.5, linewidth=3)
ax.axvline(100*512, color='k', linestyle='--', linewidth=3)
ax.text(512, 10, 'MinION', fontsize=22)
ax.text(5*512, 3*10**5, 'GridION', fontsize=22, alpha=0.5)
ax.text(3000*24, 3*10**5, 'PromethION', fontsize=22, alpha=0.5)
ax.text(100*512, 10, 'Future MinION', fontsize=22)
ax.set_ylim(5, 10**6)
ax.set_xlabel('Sequencer Channels', fontsize=22)
ax.set_ylabel('Sequencing Time', fontsize=22)
leg = ax.legend(['No Read Until', 'Guppy-lite (30W)', '1-Tile SquiggleFilter (3W)', '5-Tile SquiggleFilter (14W)'], 
                loc=(0.45,0.4), fontsize=22, labelspacing=0)
leg.legendHandles[0]._legmarker.set_markersize(9)
plt.savefig(f"{img_dir}/fig21_future-benefits.png")
plt.show()